In [14]:
import numpy as np
from cv2 import cv2
import timeit
import skimage.io as io
import matplotlib.pyplot as plt


In [15]:
#between class var maximize
def otsu_between(img):
  fimg=img.ravel()#flatten image
  total=len(fimg)#pixels
  hist=np.zeros(256)  #intensity:freq dictionary
  for i in fimg:
    hist[i]+=1
  mean=0.0 #mean of total image
  for i in range(0,256):
    mean+=i*hist[i]
  meanB=0.0
  meanF=0.0
  wB=0
  wF=0
  thresold=0
  varMax=0.0
  
  for i in range(0,256):
    wB+=hist[i]
    wF=total-wB
    meanB+=i*hist[i]
    meanF=mean-meanB
    temp=wB*wF*((meanB/wB - meanF/wF)**2)
    if(temp>varMax):
      varMax=temp
      thresold=i
  return thresold

In [16]:
#within class var minimize
def otsu_within(img):
  fimg=img.ravel()#flatten image
  total=len(fimg)#pixels
  hist=np.zeros(256)  #intensity:freq dictionary
  for i in fimg:
    hist[i]+=1
  
  within=[]
  for i in range(1,255):
    left,right=np.split(hist,[i])
    wB=np.sum(left)/total
    wF=np.sum(right)/total

    meanB=np.sum([p*q for p,q in enumerate(left)])/np.sum(left)
    meanF=np.sum([p*q for p,q in enumerate(right)])/np.sum(right)

    varB=np.sum([(p-meanB)**2 * q  for p,q in enumerate(left)])/np.sum(left)
    varF=np.sum([(p-meanF)**2 * q  for p,q in enumerate(right)])/np.sum(right)
    varB=np.nan_to_num(varB)
    within.append(varB*wB+varF*wF)
  thresold=np.argmin(within)
  return thresold

In [17]:
path='/content/drive/MyDrive/DIP/Assignment01/'
img_gray=io.imread(path+'input/Coins.png',as_gray=True)

def otsu(img):
  a=timeit.default_timer()
  ow=otsu_within(img)
  b=timeit.default_timer()
  c=timeit.default_timer()
  ob=otsu_between(img)
  d=timeit.default_timer()
  thresold=[ow,ob]
  time=[b-a,d-c]
  #print(f'otsu\'s withinn class variance take {b-a}\notsu\'s between class variance take {c-b} \ndiff= {(b-a)-(c-b)}')
  img1=np.where(ow>=img,255,0)
  img2=np.where(ob>=img,255,0)
  images=[img1,img2]
  th2,ret2 = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  return thresold,time,images,th2


#th2,ret2 = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#print(th2)

In [19]:
o=otsu(img_gray)
print(o[1])
plt.imsave(path+'output/otsu_within_img.png',o[2][0],cmap='gray')
plt.imsave(path+'output/otsu_bet_img.png',o[2][1],cmap='gray')

[0.1771156869999686, 0.03324977600004786]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
